In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from warnings import filterwarnings;
filterwarnings('ignore');

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading the Dataset

In [ ]:
df=pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')

In [ ]:
test

In [ ]:
df

In [ ]:
test.isnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.Geography.value_counts()

In [ ]:
df.NumOfProducts.value_counts()


In [ ]:
df.IsActiveMember.describe()

In [ ]:
df

In [ ]:
train_df=df[['CreditScore','Geography','Gender','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary','Exited']]
train_df
test_df=test[['CreditScore','Geography','Gender','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']]

In [ ]:
def colchange(train_df):
    train_df['Geography']=pd.to_numeric(train_df['Geography'].replace(['France','Spain', 'Germany'],[1,2,3]))
    train_df['Gender']=pd.to_numeric(train_df['Gender'].replace(['Male','Female'],[1,0]))
    return(train_df)
train_df=colchange(train_df)
test_df=colchange(test_df)
train_df['Exited']=pd.Categorical(train_df['Exited'])
print(test_df.info())
train_df.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
corr = train_df.corr()
# plot the heatmap
plt.figure(figsize=(20, 20))
s=sns.heatmap(corr,annot=True, cmap='crest')
corr = test_df.corr()
# plot the heatmap
plt.figure(figsize=(20, 20))
s=sns.heatmap(corr,annot=True, cmap='Blues')

In [ ]:
train_df

# Test and Train Split

In [ ]:
X=train_df.drop(columns=['Exited'])
y=(train_df[['Exited']].astype('int64'))

print(X.describe())
print(y.info())

## Scaling the Input

In [ ]:
def scale(X):
    from sklearn import preprocessing
    scaled=preprocessing.StandardScaler()
    scaler=scaled.fit(X)
    X=scaler.transform(X)
    return X

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(scale(X),y,train_size=0.8, random_state=42)
X.keys()
test_df=scale(test_df)

In [ ]:
y.info()

In [ ]:
def evaluate(y_test,ypred):
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    print("Accuracy: ",accuracy_score(y_test,y_pred)) 
    print("Precision Score : ", precision_score(y_test,y_pred)) #precision measures the proportion of true positive predictions among all positive instances. how many of survived predicted actually survived, doesn't verifies 0's 70 survived as preicted whereas actually 92 survived so 70/92 will be the precision.  if we predicted 70 survived, so presion will tell how many of those 70 predicted survived matches the actual row by row data. It checkes all positives and verifies if the answer is true for each row?
    print("Recall Score: ", recall_score(y_test,y_pred, average='macro')) #Recall measures the proportion of true positive predictions among all actual positive instalnces. If we predicted 100 survived correctly whereas actually 100 survived out of which 67 predicted correctly so recall will be 0.67
    print("F1 Score: ",f1_score(y_test,y_pred)) #mean of recall and precision
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 10))
    s=sns.heatmap(cm,annot=True, cmap='Reds')


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
result=pd.DataFrame(columns=['id', 'RF','MLP','XGB','ABC','GBC','Exited'])
result.id=test[['id']]
weight={'MLP':0,'RF':0,'XGB':0, 'ABC':0,'GBC':0}
models = {
   'RF': RandomForestClassifier(),
   'XGB' : xgb.XGBClassifier(objective="binary:logistic",seed=42),
   'ABC': AdaBoostClassifier(),
   'GBC': GradientBoostingClassifier(validation_fraction=0.1, n_iter_no_change=15),
   'MLP':MLPClassifier() 
}
grid_param={
   'MLP': {
       'solver': ['lbfgs', 'adam'], 
       'early_stopping':[1],
       'activation': ['tanh', 'relu'], 
       'max_iter': [500], 
       'alpha': [10.0 ** -8], 
       'hidden_layer_sizes':[13,12,14], 
       'learning_rate': ['adaptive'],
       'verbose':[0] 
          },
   'RF' : {
       'bootstrap': [True], 
       'ccp_alpha': [0,0.1,0.01], 
       'max_depth': [None], 
       'max_leaf_nodes': [None], 
       'max_samples': [None], 
       'min_impurity_decrease': [0.0, 0.1, 0.01], 
       'min_samples_leaf': [1], 
       'min_samples_split': [4], 
       'n_estimators': [110,105,120], 
       'warm_start': [True] 
        },
   'XGB': {
        'n_estimators':[30, 100, 1000],
        'learning_rate': [0.1, 0.01, 0.05]
          },
   'GBC': {
        'loss':['log_loss', 'exponential'],
        'criterion': ['friedman_mse', 'squared_error'],
        'n_estimators': [10, 1000, 100],
        'learning_rate': [0.1,0.04,0.01],
        'min_impurity_decrease':[0.1,0.01],
        'ccp_alpha':[0.03,0.07,0.05],
        'subsample': [1.5, 1, 2]
        },
  'ABC':  {
        'n_estimators':[50,100, 1000],
        'learning_rate':[0.1,0.01, 0.001]
        },
}
for name,mod in tqdm(models.items()):
   gs = GridSearchCV(estimator=mod, param_grid=grid_param[name], cv=3, scoring='accuracy')
   gs.fit(X_train,np.ravel(y_train))
   print('Model: ', name, ' Accuracy:',format(gs.best_score_,".2f"),  ' Best Parameters:',gs.best_params_)
   y_pred=gs.predict(X_test)
   weight[name]= format(gs.best_score_,".2f")
   result[name]=gs.predict_proba(scale(test_df))[:,1]
   
   evaluate(y_test,y_pred)
# Model:  DT  Accuracy: 0.80  Best Parameters: {'ccp_alpha': 0.0, 'max_depth': None, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 4, 'random_state': 42}
#Model:  RF  Accuracy: 0.86  Best Parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'max_depth': None, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 110, 'warm_start': True}
#Model:  MLP  Accuracy: 0.86  Best Parameters: {'activation': 'relu', 'alpha': 1e-08, 'early_stopping': 1, 'hidden_layer_sizes': 13, 'learning_rate': 'adaptive', 'max_iter': 200, 'random_state': 9, 'solver': 'lbfgs', 'verbose': 0}

In [ ]:
weight

In [ ]:
result

## Result Submission

In [ ]:
result['Exited']= np.round((((weight.get('RF')* result.RF) + (weight.get('MLP')*result.MLP) + (weight.get('XGB')*result.XGB) + (weight.get('GBC')*result.GBC)+ (weight.get('ABC')*result.ABC))/(5*max(weight.values()))),2)

result

In [ ]:
submission=result[['id','Exited']]
submission.to_csv('submission.csv', index=False)

In [ ]:
submission